In [ ]:
import pandas as pd
import numpy as np

Необходимо привести данные к виду:
|Имя дисциплины|Семестр|Посещаемость лекций 1 пром|Посещаемость лекций 2 пром|Посещаемость лекций 3 пром|Посещаемость практик 1 пром|Посещаемость практик 2 пром|Посещаемость практик 3 пром|КТ 1|КТ 2|Экзамен|Балл ктр|Балл лаб|
|-|-|-|-|-|-|-|-|-|-|-|-|-|
|Имя|Семестр|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|[0,1]|1|1|

# Реализация

Берем таблицу attendance-students

In [ ]:
attendance_students = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-students.csv').rename(columns={'id':'attendance_id', 'contingent_id':'connection_id'})

attendance_students.head(5)

Берем таблицу ocenka-students

In [ ]:
ocenka_students = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-students.csv').rename(columns={'id':'ocenka_id', 'study_id':'connection_id'})

ocenka_students.head(5)

Добавляем эти таблицы в мастер-таблицу

In [ ]:
master_table = attendance_students.set_index('connection_id').join(ocenka_students.set_index('connection_id')).dropna()

master_table.head(5)

Удаляем из таблицы все кроме id

In [ ]:
master_table.drop(columns=['enrollment_at', 'deleted_at'], axis=1, inplace=True)

master_table.head(5)

Добавляем в таблицу группы и удаляем все, кроме id студентов и групп.

В дальнейшем в принципе будет удаляться все, кроме интересующей информации, даже если придется её потом снова добавлять

In [ ]:
ocenka_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-groups.csv').rename(columns={'id':'group_id'})

ocenka_groups.head(5)

In [ ]:
master_table = master_table.join(ocenka_groups.rename(columns={'number':'group_num'}).set_index('group_id'), on='group_id').dropna()

master_table.drop(['group_id', 'edu_program_full_title'], axis=1, inplace=True)

master_table.head(5)

Получим id групп со всех баз

In [ ]:
attendance_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-groups.csv').rename(columns={'id':'group_id'})

attendance_groups.head(5)

In [ ]:
timetable_groups = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-groups.csv').rename(columns ={'id':'group_id'})

timetable_groups.head(5)

In [ ]:
master_table = master_table.join(ocenka_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_ocenka'}).set_index('group_num'), on='group_num').dropna()
master_table = master_table.join(attendance_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_attendance'}).set_index('group_num'), on='group_num').dropna()
master_table = master_table.join(timetable_groups[['group_id', 'number']].rename(columns={'number':'group_num', 'group_id':'group_id_timetable'}).set_index('group_num'), on='group_num').dropna()

master_table = master_table.reset_index()

master_table.head(10)

Добавляем данные о семестре

In [ ]:
timetable_courses = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-courses.csv').rename(columns={'id':'course_id'})

timetable_courses.head(5)

In [ ]:
timetable_faculties = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-faculties.csv').rename(columns={'id':'faculty_id'})

timetable_faculties.head(5)

In [ ]:
buffer_table = timetable_courses[['course_id', 'faculty_id']]

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.join(timetable_faculties[['faculty_id', 'semester_id']].set_index('faculty_id'), on='faculty_id').dropna()

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.set_index(['course_id', 'faculty_id']).sub(buffer_table['semester_id'].min()).reset_index()[['course_id', 'semester_id']]

buffer_table.head(10)

In [ ]:
master_table = master_table.join(timetable_groups[['group_id', 'course_id']].rename(columns={'group_id':'group_id_timetable'}).set_index('group_id_timetable'), on='group_id_timetable').dropna()

master_table.head(10)

In [ ]:
master_table = master_table.join(buffer_table.set_index('course_id'), on='course_id').dropna()

master_table.head(5)

In [ ]:
master_table.drop(['course_id'], axis=1,inplace=True)
master_table = master_table.rename(columns={'semester_id':'semester'})

master_table.head(5)

Находим все дисциплины группы

In [ ]:
timetable_educations = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-educations.csv').rename(columns={'id':'edu_id'})

timetable_educations.head(5)

In [ ]:
timetable_disciplines = pd.read_csv('C:\\MyFolders\\ASU2304\\TEST\\data\\timetable\\timetable-disciplines.csv').rename(columns={'id':'discipline_id'})

timetable_disciplines.head(5)

In [ ]:
buffer_table = timetable_educations

buffer_table = buffer_table.join(timetable_disciplines[['discipline_id', 'name']].rename(columns={'name':'names'}).set_index('discipline_id'), on='discipline_id').dropna()

buffer_table.head(5)

In [ ]:
master_table = master_table.join(buffer_table[['group_id', 'discipline_id', 'names']].rename(columns={'discipline_id':'discipline_id_timetable', 'names':'discipline_name_timetable'}).set_index('group_id'), on='group_id_timetable').dropna().reset_index()

master_table.drop(['index'], axis=1,inplace=True)

master_table.head(5)

Считаем посещаемость

In [ ]:
attendance_lessons = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-lessons.csv")

attendance_lessons.head(5)

In [ ]:
attendance_disciplines = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-disciplines.csv")

attendance_disciplines.head(5)

In [ ]:
attendance_presences = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\attendance\\attendance-presences.csv")

attendance_presences['state'].fillna('wasnt', inplace=True)

attendance_presences.head(5)

In [ ]:
buffer_table = attendance_lessons[['id', 'group_id', 'discipline_id', 'date_on', 'kind']].loc[((attendance_lessons['kind'] == 'lecture') |(attendance_lessons['kind'] == 'practice'))]
buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.join(attendance_presences[['student_id', 'lesson_id', 'state']].set_index('lesson_id'), on='id').rename(columns={'date_on':'date'}).dropna().reset_index()

buffer_table.drop(['index', 'id'], axis=1, inplace=True)

buffer_table.head()

In [ ]:
counter = 0
for i in range(2018,2030):
    buffer_table.loc[(buffer_table['date'] >= f'{i}-09-01') & (buffer_table['date'] < f'{i}-10-16'), 'period'] = 1
    buffer_table.loc[(buffer_table['date'] >= f'{i}-10-16') & (buffer_table['date'] < f'{i}-12-04'), 'period'] = 2
    buffer_table.loc[(buffer_table['date'] >= f'{i}-12-04') & (buffer_table['date'] < f'{i+1}-01-15'), 'period'] = 3
    
    buffer_table.loc[(buffer_table['date'] >= f'{i}-09-01') & (buffer_table['date'] <= f'{i+1}-01-15'), 'semester'] = counter
    counter+=1
    
    
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-02-03') & (buffer_table['date'] < f'{i+1}-03-15'), 'period'] = 1
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-03-15') & (buffer_table['date'] < f'{i+1}-04-15'), 'period'] = 2
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-04-15') & (buffer_table['date'] <= f'{i+1}-06-30'), 'period'] = 3
    
    buffer_table.loc[(buffer_table['date'] >= f'{i+1}-02-03') & (buffer_table['date'] <= f'{i+1}-06-30'), 'semester'] = counter
    counter+=1

buffer_table.dropna(inplace=True)

buffer_table.drop('date', axis=1, inplace=True)

buffer_table.head(5)

In [ ]:
buffer_table.loc[buffer_table['state'] == 'was', 'state'] = 1
buffer_table.loc[buffer_table['state'] == 'wasnt', 'state'] = 0

buffer_table = buffer_table.reset_index()

buffer_table.drop('index', axis=1, inplace=True)

buffer_table.head(5)

In [ ]:
sum = buffer_table
sum = sum.groupby(['group_id', 'discipline_id', 'kind', 'student_id', 'period', 'semester'])['state'].sum().reset_index().rename(columns={'state':'count_presences'})

sum.head(5)

In [ ]:
count = buffer_table
count = count.groupby(['group_id', 'discipline_id', 'kind', 'student_id', 'period', 'semester'])['state'].count().reset_index().rename(columns={'state':'count_lessons'})

count.head(5)

In [ ]:
buffer_table = sum.join(count[['count_lessons']]).dropna()

buffer_table.head(5)

In [ ]:
buffer_table['attendance'] = buffer_table['count_presences'] / buffer_table['count_lessons']

buffer_table.drop('count_presences', axis=1, inplace=True)
buffer_table.drop('count_lessons', axis=1, inplace=True)

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.join(attendance_disciplines.set_index('id'), on='discipline_id').dropna()

buffer_table.drop('discipline_id', axis=1, inplace=True)
buffer_table.drop('group_id', axis=1, inplace=True)
buffer_table = buffer_table.rename(columns={'student_id':'attendance_id'})

buffer_table.head(5)

In [ ]:
master_table = master_table.join(buffer_table.set_index(['attendance_id', 'semester', 'title']), on=['attendance_id', 'semester', 'discipline_name_timetable']).dropna().rename(columns = {'discipline_name_timetable':'discipline_name'}).drop('discipline_id_timetable', axis=1)

master_table.head(5)

Собираем оценки

In [ ]:
ocenka_semesters = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-semesters.csv").rename(columns={'id':'semester_id'})

ocenka_semesters.head()

In [ ]:
ocenka_discipline_semesters = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-discipline-semesters.csv").rename(columns={'id':'discipline_semester_id'})

ocenka_discipline_semesters.head(5)

In [ ]:
ocenka_disciplines = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-disciplines.csv").rename(columns={'id':'discipline_id'})

ocenka_disciplines.head(5)

In [ ]:
ocenka_marks = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-marks.csv").rename(columns={'id':'mark_id'})

ocenka_marks.head(5)

In [ ]:
ocenka_discipline_semesters_periods = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\ocenka\\ocenka-discipline-semester-periods.csv").rename(columns={'id':'discipline_semester_period_id'})

ocenka_discipline_semesters_periods.head(5)

In [ ]:
buffer_table = ocenka_semesters

buffer_table.head(5)

In [ ]:
buffer_table.drop(['kind', 'academic_year'], axis=1, inplace=True)

buffer_table['semester_num'] = buffer_table['semester_id'] - 11

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.join(ocenka_discipline_semesters.set_index('semester_id'), on='semester_id').dropna()

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.join(ocenka_disciplines.set_index('discipline_id'), on='discipline_id').drop('abbr', axis=1).dropna()

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.join(ocenka_discipline_semesters_periods.set_index('discipline_semester_id'), on='discipline_semester_id').dropna()

buffer_table.head(5)

In [ ]:
buffer_table = buffer_table.drop(['semester_id', 'discipline_semester_id', 'discipline_id', 'period_id', 'group_id'], axis=1).join(ocenka_marks.drop(['mark_id', 'marked_on', 'passed_on'], axis=1).set_index('discipline_semester_period_id'), on='discipline_semester_period_id').dropna().drop('discipline_semester_period_id', axis=1)

buffer_table.head(5)

In [ ]:
buffer_table.rename(columns={'semester_num':'semester', 'title':'discipline'}, inplace=True)
buffer_table.reset_index(inplace=True)
buffer_table.drop('index', axis=1, inplace=True)

buffer_table.head(5)

In [ ]:
buffer_table.loc[(buffer_table['kind'] == 'exam') | (buffer_table['kind'] == 'qualification') | (buffer_table['kind'] == 'diff_qualification') | (buffer_table['kind'] == 'qualification_with_mark'), 'period'] = 3
buffer_table.loc[(buffer_table['kind'] == 'kt_2'), 'period'] = 2
buffer_table.loc[(buffer_table['kind'] == 'kt_1'), 'period'] = 1
buffer_table.loc[(buffer_table['kind'] == 'course_projecting'), 'period'] = None

buffer_table.drop('kind', axis=1, inplace=True)
buffer_table.dropna()

buffer_table.head(5)

In [ ]:
master_table = master_table.join(buffer_table.rename(columns={'student_id':'ocenka_id'}).set_index(['semester', 'discipline', 'ocenka_id', 'period']), on=['semester', 'discipline_name', 'ocenka_id', 'period']).reset_index().dropna()
master_table['grade'] = master_table['grade']/5

master_table.head(5)

Создаем финальную таблицу, которая будет сформирована, как пример в начале

In [ ]:
master_table.drop(['attendance_id', 'ocenka_id', 'group_num', 'group_id_ocenka', 'group_id_attendance', 'group_id_timetable'], axis=1, inplace=True)

master_table.head(5)

In [ ]:
final_table = pd.get_dummies(master_table, columns=['period', 'kind'])

final_table.head(5)

In [ ]:
final_table.loc[(final_table['period_1.0'] == True) & (final_table['kind_lecture'] == True), 'lecture_1'] = True
final_table.loc[(final_table['period_2.0'] == True) & (final_table['kind_lecture'] == True), 'lecture_2'] = True
final_table.loc[(final_table['period_3.0'] == True) & (final_table['kind_lecture'] == True), 'lecture_3'] = True

final_table.loc[(final_table['period_1.0'] == True) & (final_table['kind_practice'] == True), 'practice_1'] = True
final_table.loc[(final_table['period_2.0'] == True) & (final_table['kind_practice'] == True), 'practice_2'] = True
final_table.loc[(final_table['period_3.0'] == True) & (final_table['kind_practice'] == True), 'practice_3'] = True

final_table.loc[(final_table['period_1.0'] == True), 'kt_1'] = True
final_table.loc[(final_table['period_2.0'] == True), 'kt_2'] = True
final_table.loc[(final_table['period_3.0'] == True), 'exam'] = True

final_table.drop(['period_1.0', 'period_2.0', 'period_3.0', 'kind_lecture', 'kind_practice'], axis=1, inplace=True)

final_table.fillna(False, inplace=True)

final_table.head(5)

In [ ]:
def move_attendance(row):
    for col in ['lecture_1', 'lecture_2', 'lecture_3', 'practice_1', 'practice_2', 'practice_3']:
        if row[col]:
            row[col] = row['attendance']
        else:
            row[col] = 0
    return row

def move_grade(row):
    for col in ['kt_1', 'kt_2', 'exam']:
        if row[col]:
            row[col] = row['grade']
        else:
            row[col] = 0
    return row

final_table = final_table.apply(move_attendance, axis=1).drop(['index', 'attendance'], axis=1)
final_table = final_table.apply(move_grade, axis=1).drop(['grade'], axis=1)

final_table.head(15)

In [ ]:
one = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['lecture_1'].sum().reset_index()
two = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['lecture_2'].sum().reset_index()
three = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['lecture_3'].sum().reset_index()
four = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['practice_1'].sum().reset_index()
five = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['practice_2'].sum().reset_index()
six = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['practice_3'].sum().reset_index()

seven = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['kt_1'].sum().reset_index()
eight = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['kt_2'].sum().reset_index()
nine = final_table.groupby(['connection_id', 'semester', 'discipline_name'])['exam'].sum().reset_index()

final_table = one.join(two.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])
final_table = final_table.join(three.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])
final_table = final_table.join(four.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])
final_table = final_table.join(five.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])
final_table = final_table.join(six.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])

final_table = final_table.join(seven.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])
final_table = final_table.join(eight.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])
final_table = final_table.join(nine.set_index(['connection_id', 'semester', 'discipline_name']), on=['connection_id', 'semester', 'discipline_name'])

final_table.head(5)

In [ ]:
def normalize(row):
    for col in ['kt_1', 'kt_2', 'exam']:
        row[col] /= 2
    return row

final_table = final_table.apply(normalize, axis=1)

final_table.head(5)

In [ ]:
def set_def_values(row):
    for col in ['controls', 'labs']:
        row[col] = 1
    return row

final_table = final_table.apply(set_def_values, axis=1)

final_table.head(5)

Сохраняем файл

In [ ]:
final_table = final_table.reset_index().drop('index', axis=1)

final_table.to_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\final\\data.csv")

ЭЭЭЭЭКСПЕРИМЕНТЫ